<a href="https://colab.research.google.com/github/shubhangkhare/Learnings/blob/main/Question_Answering_using_Spacy_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **spaCy meets Transformers: Fine-tune BERT, XLNet and GPT-2**
Read blog: https://explosion.ai/blog/spacy-transformers

Trained Pipelines:
1. en_core_web_sm: English pipeline optimized for CPU.
2. en_core_web_md: English pipeline optimized for CPU.
3. en_core_web_lg: English pipeline optimized for CPU.
4. en_core_web_trf: English transformer pipeline (roberta-base).

spaCy English pipelines: https://spacy.io/models/en#en_core_web_trf

# Dependencies and Libraries

### Install and download
Before you install spaCy and its dependencies, make sure that your pip, setuptools and wheel are up to date.

In [25]:
!pip install -U pip setuptools wheel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [1]:
import wheel

In [2]:
!pip install spacy-transformers
# Below code is not working
!python -m spacy download en_core_web_trf # transformers Roberta-base

2023-06-29 18:27:55.958386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 18:27:57.466474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 2.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


https://github.com/explosion/spacy-models/releases/tag/en_core_web_trf-3.5.0

In [15]:
# Fix for above code
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.5.0/en_core_web_trf-3.5.0.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 964.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en-core-web-trf: filename=en_core_web_trf-3.5.0-py3-none-any.whl size=460267966 sha256=8084ac3281c19ce6482fa54383009831a7e1a318fe9340878350d5fc1c3fc814
  Stored in directory: /root/.cache/pip/wheels/f3/ee/73/36aeb439d17c95a9c57bf8df6fb67e0cebbd842270ae3d501e
Successfully built en-core-web-trf


### Import Libraries

In [3]:
import spacy
import torch
import numpy
from numpy.testing import assert_almost_equal
import pandas as pd

### GPU configuration and load spacy

In [25]:
#is_using_gpu = spacy.prefer_gpu()
is_using_gpu = None
if is_using_gpu:
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

nlp = spacy.load("en_core_web_trf")

In [26]:
type(nlp)

spacy.lang.en.English

In [27]:
nlp("Here is some text to encode.")

Here is some text to encode.

In [16]:
'''doc = nlp("Here is some text to encode.")
#assert doc.tensor.shape == (7, 768)  # Always has one row per token
doc._.trf_word_pieces_  # String values of the wordpieces
doc._.trf_word_pieces  # Wordpiece IDs (note: *not* spaCy's hash values!)
doc._.trf_alignment  # Alignment between spaCy tokens and wordpieces
# The raw transformer output has one row per wordpiece.
assert len(doc._.trf_last_hidden_state) == len(doc._.trf_word_pieces)
# To avoid losing information, we calculate the doc.tensor attribute such that
# the sum-pooled vectors match (apart from numeric error)
assert_almost_equal(doc.tensor.sum(axis=0), doc._.trf_last_hidden_state.sum(axis=0), decimal=5)
span = doc[2:4]
# Access the tensor from Span elements (especially helpful for sentences)
assert numpy.array_equal(span.tensor, doc.tensor[2:4])
# .vector and .similarity use the transformer outputs'''


apple1 = nlp("My name is Shubhang Khare and I am a student")
apple2 = nlp("Apple sold fewer iPhones this quarter.")
apple3 = nlp("Apple pie is delicious.")
print(apple1[0].similarity(apple2[0]))  # 0.73428553
print(apple1[0].similarity(apple3[0]))  # 0.43365782

AttributeError: ignored

In [11]:
apple1.vector

array([], dtype=float32)

In [24]:
print(nlp("Banane").similarity(nlp("Fruit")))

0.0


<ipython-input-24-c42f7e5e7192>:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(nlp("Banane").similarity(nlp("Fruit")))
<ipython-input-24-c42f7e5e7192>:1: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(nlp("Banane").similarity(nlp("Fruit")))


# Load Dataset

### Real Data

In [2]:
data = pd.read_csv('your_file.csv')
questions = data['Question'].tolist()
answers = data['Answer'].tolist()

NameError: ignored

### Sample Data

In [ ]:
questions = [
    "What is the capital of India?",
    "What is the official language of India?",
    "Who is the Prime Minister of India?",
    "What is the currency of India?",
    "What is the population of India?",
    "What are the major religions in India?",
    # Add more questions...
]

answers = [
    "New Delhi",
    "Hindi",
    "Narendra Modi",
    "Indian Rupee",
    "1.3 billion",
    "Hinduism, Islam, Christianity, Sikhism, Buddhism, Jainism",
    # Add more answers...
]

data = pd.DataFrame({"Question": questions, "Answer": answers})
# data.to_csv("india_qa.csv", index=False)
questions = data['Question'].tolist()
answers = data['Answer'].tolist()

In [ ]:
data.head()

# Process the data with SpaCy
Next, you'll need to process the questions and answers using SpaCy. This involves tokenization, part-of-speech tagging, and dependency parsing. You can use the pre-trained English model en_core_web_sm for this.

In [ ]:
nlp = spacy.load('en_core_web_md')

processed_questions = [nlp(q) for q in questions]
processed_answers = [nlp(a) for a in answers]

# Implement the question answering logic
To answer a given question, you'll need to find the most relevant answer from your processed answers. One way to do this is by calculating the similarity between the question and each answer using SpaCy's similarity method. You can then choose the answer with the highest similarity score.

In [ ]:
def get_most_similar_answer(question):
    question_tokens = nlp(question)
    max_similarity = -1
    most_similar_answer = None

    for answer, processed_answer in zip(answers, processed_answers):
        similarity = question_tokens.similarity(processed_answer)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_answer = answer

    return most_similar_answer

In [ ]:
def get_most_similar_answer(user_ques):
    user_ques_emb = nlp(user_ques)
    most_similar_answer = None
    count = 0

    for q, a in zip(questions, answers):
        data_ques_emb = nlp(q)
        similarity = user_ques_emb.similarity(data_ques_emb)

        if count == 0:
            max_similarity = similarity
            most_similar_answer = a

        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_answer = a

        count += 1
    return most_similar_answer

# Test the question answering system

In [ ]:
question = "Can you tell me India's Capital"
answer = get_most_similar_answer(question)
print(answer)